In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten


In [ ]:
dataset=pd.read_csv('../input/spaceship-titanic/train.csv')
test=pd.read_csv('../input/spaceship-titanic/test.csv')
#dataset.info()
dataset=dataset.dropna()
test_data=test.copy()

one_hot=LabelEncoder()
dataset['CryoSleep']=one_hot.fit_transform(dataset['CryoSleep'])
dataset['VIP']=one_hot.fit_transform(dataset['VIP'])
dataset['HomePlanet']=one_hot.fit_transform(dataset['HomePlanet'])
dataset['Destination']=one_hot.fit_transform(dataset['Destination'])
dataset['Cabin']=one_hot.fit_transform(dataset['Cabin'])
dataset['Transported']=one_hot.fit_transform(dataset['Transported'])
print(dataset.head())

test_data['CryoSleep']=one_hot.fit_transform(test_data['CryoSleep'])
test_data['VIP']=one_hot.fit_transform(test_data['VIP'])
test_data['HomePlanet']=one_hot.fit_transform(test_data['HomePlanet'])
test_data['Destination']=one_hot.fit_transform(test_data['Destination'])
test_data['Cabin']=one_hot.fit_transform(test_data['Cabin'])



In [ ]:
cols=['FoodCourt','ShoppingMall']
dataset['new']=0
test_data['new']=0
for name in cols:
    dataset['new']+=dataset[name]
    test_data['new']+=test_data['new']

cols=['Spa','VRDeck','RoomService']
dataset['new1']=0
test_data['new1']=0
for name in cols:
    dataset['new1']+=dataset[name]
    test_data['new1']+=test_data[name]


corr_matrix=dataset.corr()
print(corr_matrix["Transported"].sort_values(ascending=False))


y=dataset['Transported']
dataset=dataset.drop(['VIP','Cabin','Age','Transported','Name','PassengerId'],axis=1)
test_data=test_data.drop(['VIP','Cabin','Age','Name','PassengerId'],axis=1)
dataset.info()
print(dataset.head())

cols=['Spa','VRDeck','RoomService','FoodCourt','ShoppingMall','new','new1']
for column in cols:
     dataset[column] = dataset[column] /dataset[column].abs().max()
print(dataset.head())


cols=['Spa','VRDeck','RoomService','FoodCourt','ShoppingMall','new','new1']
for column in cols:
     test_data[column] = test_data[column] /test_data[column].abs().max()
test_data=pd.DataFrame(dataset)

train_x,valid_x,train_y,valid_y=train_test_split(dataset,y,test_size=0.20,random_state=24)
print(train_x.shape)


model=keras.Sequential()
model.add(Dense(64,activation='relu',input_shape=((5284, 10))))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)
history=model.fit(train_x,train_y,validation_data=(valid_x,valid_y),batch_size=64,epochs=200,callbacks=[early_stopping],verbose=0)

history_df = pd.DataFrame(history.history)
# Start to plot at epoch 5
history_df.loc[5:, ['loss', 'val_loss']].plot()
history_df.loc[5:, ['accuracy', 'val_accuracy']].plot()
plt.show()

print(("Best Loss: {:0.4f}" + "\nBest Accuracy: {:0.4f}")\
        .format(history_df['loss'].min(),history_df['accuracy'].max()))
print(("Best Validation Loss: {:0.4f}" + "\nBest Validation Accuracy: {:0.4f}")\
        .format(history_df['val_loss'].min(),history_df['val_accuracy'].max()))



In [ ]:
ids=test['PassengerId']
test_pred=model.predict(test_data)
test_pred=[round(x[0]) for x in test_pred]
test_pred=list(zip(ids,test_pred))
result=pd.DataFrame(test_pred,columns=['PassengerId','Transported'])
result=result.set_index('PassengerId').to_csv('./pred.csv')